In [4]:
import pandas as pd#
import requests
import bs4

In [ ]:

headers = {'Accept' : "*/*",
'Accept-Language' : "en-US,en;q=0.9'",
'Connection' : "keep-alive'",
'Content-Type' : "application/x-www-form-urlencoded; charset=UTF-8'",
'Cookie' : "_gid=GA1.2.1744189115.1668267751; PHPSESSID=lrnamabi5qtnm6ipq8s5b35ipk; _ga_CR24R4W7G3=GS1.1.1668267750.1.1.1668267883.0.0.0; trc_cookie_storage=taboola%2520global%253Auser-id%3Dead09a75-9bd9-40d9-b735-84d7bf166307-tucta3b7fa5; _ga=GA1.2.531214221.1668267751; _ga_HBTE4JTNMB=GS1.1.1668267884.1.1.1668269233.0.0.0'",
'Origin' : "https://www.football-lineups.com",
'Referer' : "https://www.football-lineups.com/season/Liverpool/2022-2023/",
'Sec-Fetch-Dest' : "empty'",
'Sec-Fetch-Mode' : "cors'",
'Sec-Fetch-Site' : "same-origin'",
'User-Agent' : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'",
'X-Requested-With' : "XMLHttpRequest'",
'sec-ch-ua' : '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
'sec-ch-ua-mobile' : "?0'",
'sec-ch-ua-platform' : "Linux"
  }
#     --data-raw 'd1=25&d2=-2022&d3=0&d4=1&d5=0&d6=1&d7=20&d8=0&d9=&d10=0&d11=0&d12=undefined' \
#   --compressed

In [37]:
url="https://www.football-lineups.com/ajax/get_sectf.php"

headers = {
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    "sec-ch-ua": "\"Google Chrome\";v=\"107\", \"Chromium\";v=\"107\", \"Not=A?Brand\";v=\"24\"",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "x-requested-with": "XMLHttpRequest",
    "cookie": "_gid=GA1.2.1744189115.1668267751; PHPSESSID=lrnamabi5qtnm6ipq8s5b35ipk; _ga_CR24R4W7G3=GS1.1.1668267750.1.1.1668267883.0.0.0; trc_cookie_storage=taboola%2520global%253Auser-id%3Dead09a75-9bd9-40d9-b735-84d7bf166307-tucta3b7fa5; _ga=GA1.2.531214221.1668267751; _ga_HBTE4JTNMB=GS1.1.1668267884.1.1.1668269233.0.0.0",
    "Referer": "https://www.football-lineups.com/season/Liverpool/2022-2023/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
  }
body= "d1=25&d2=-2022&d3=0&d4=1&d5=0&d6=1&d7=20&d8=0&d9=&d10=0&d11=0&d12=undefined"

In [43]:
payload = {
"d1": "25",
"d2": "-2022",
"d3": "0",
"d4": "1",
"d5": "0",
"d6": "1",
"d7": "20",
"d8": "0",
"d9": """",
"d10": "0",
"d11": "0"}

In [44]:
res = requests.post(url, headers=headers, data=body)

In [45]:
res.text

'<html>\n<head>\n<META NAME="ROBOTS" CONTENT="NOINDEX">\n</head>\n<body>\n\n\n</body>\n</html>\n\n'

In [7]:
soup = bs4.BeautifulSoup(res.text)

In [16]:
fixtures =soup.find(id="sptf")

In [29]:
soup.find_all("div")

[<div id="dek" style="position: absolute; top:5px;left:300px; visibility: hidden; z-index: 200;"></div>,
 <div id="dek2" style="position: absolute; top:5px;left:300px; visibility: hidden; z-index: 199;"></div>,
 <div id="snack_ldb"></div>,
 <div class="dropdown_2columns"><div class="col_2"><span id="spmenu30_na"></span></div></div>,
 <div class="col_2"><span id="spmenu30_na"></span></div>,
 <div class="dropdown_2columns"><div class="col_2"> <span id="spmenu30_gb"></span> </div></div>,
 <div class="col_2"> <span id="spmenu30_gb"></span> </div>,
 <div class="dropdown_4columns"><div class="col_4"><span id="spmenu30_eu"></span></div></div>,
 <div class="col_4"><span id="spmenu30_eu"></span></div>,
 <div class="dropdown_5columns"><div class="col_5"><span id="spmenu30_am"></span></div></div>,
 <div class="col_5"><span id="spmenu30_am"></span></div>,
 <div class="dropdown_1column align_right"><div class="col_1"><ul class="simple"><span id="spmenu30_rc"></span></ul></div></div>,
 <div class="c